<a href="https://colab.research.google.com/github/dharma610/Project/blob/master/SIR_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from scipy.integrate import odeint
import numpy as np
import pandas as pd
from random import sample
import matplotlib.pyplot as plt
%matplotlib inline 
!pip install mpld3
import mpld3
mpld3.enable_notebook()

     |████████████████████████████████| 1.0MB 2.9MB/s 
  Created wheel for mpld3: filename=mpld3-0.5.1-cp36-none-any.whl size=364065 sha256=ebb81a02167c126f5e47756bf385a7575621baa2f3c1aefb8d6f798689ec67d2
  Stored in directory: /root/.cache/pip/wheels/38/68/06/d119af6c3f9a2d1e123c1f72d276576b457131b3a7bf94e402
Successfully built mpld3


In [2]:
def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

In [3]:
def plotsir(t, S, I, R):
  f, ax = plt.subplots(1,1,figsize=(10,4))
  ax.plot(t, S, 'b', alpha=0.7, linewidth=2, label='Susceptible')
  ax.plot(t, I, 'y', alpha=0.7, linewidth=2, label='Infected')
  ax.plot(t, R, 'g', alpha=0.7, linewidth=2, label='Recovered')
  ax.set_xlabel('Time (days)')
  ax.yaxis.set_tick_params(length=0)
  ax.xaxis.set_tick_params(length=0)
  ax.grid(b=True, which='major', c='w', lw=2, ls='-')
  legend = ax.legend()
  legend.get_frame().set_alpha(0.5)
  for spine in ('top', 'right', 'bottom', 'left'):
      ax.spines[spine].set_visible(False)
  plt.show();

In [4]:
""" gamma is a list of 1000 random number in (0,1) """

import random
gamma = []
for i in range(1000):
    gamma.append(random.uniform(0, 1))
    


In [5]:
""" beta is twice of each of 1000 gamma values """
beta = []
for i in range(1000):
  beta.append(2*gamma[i])  

In [6]:
""" P, Q, M are the list of S, I, R respectievely calculated for 100 days for each of  1000 sets of parameter (beta,gamma)."""
    
P = []
Q = []
M  = []
for i in range(1000): #Plotted for 2 sets
  N = 1000
  S0, I0, R0 = 999, 1, 0  # initial conditions: one infected, rest susceptible
  t = np.linspace(0, 99, 100) # Grid of time points (in days)
  y0 = S0, I0, R0 # Initial conditions vector
  # Integrate the SIR equations over the time grid, t.
  ret = odeint(deriv, y0, t, args=(N, beta[i], gamma[i]))
  S, I, R = ret.T
  P.append(S)
  Q.append(I)
  M.append(R)

  

In [7]:
""" list to dataftrame """

S_value = pd.DataFrame(P)
I_value = pd.DataFrame(Q)
R_value = pd.DataFrame(M)

In [8]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing.sequence import TimeseriesGenerator
# define dataset
s_list = []
i_list = []
r_list = []
s_list1 = []
i_list1= []
r_list1= []
for i in range(1000):
  s_list.append(array(S_value.iloc[i][0:70]))
  i_list.append(array(I_value.iloc[i][0:70]))
  r_list.append(array(R_value.iloc[i][0:70]))


for i in range(1000):
  s_list1.append(array(S_value.iloc[i][70:]))
  i_list1.append(array(I_value.iloc[i][70:]))
  r_list1.append(array(R_value.iloc[i][70:]))  

In [9]:
# reshape series
for i in range(1000):
  s_list[i] =s_list[i].reshape(70,1)
  i_list[i] =i_list[i].reshape(70,1)
  r_list[i] =r_list[i].reshape(70,1)

for i in range(1000):
  s_list1[i] =s_list1[i].reshape(30,1)
  i_list1[i] =i_list1[i].reshape(30,1)
  r_list1[i] =r_list1[i].reshape(30,1)  

In [11]:
p = []
q = []
r = []
p1 = []
q1 = []
r1 = []
for i in range(1000):
  p.append(s_list[i])
  q.append(i_list[i])
  r.append(r_list[i])
for i in range(1000):
  p1.append(s_list1[i])
  q1.append(i_list1[i])
  r1.append(r_list1[i])  



In [12]:
s = tuple(p)
t = tuple(q)
u = tuple(r)

s1 = tuple(p1)
t1 = tuple(q1)
u1 = tuple(r1)

In [13]:
# horizontally stack columns
dataset = hstack(s)
# define generator
n_input = 2
n_features = dataset.shape[1]
generator = TimeseriesGenerator(dataset, dataset, length=n_input, batch_size=1)
# number of samples
print('Samples: %d' % len(generator))

Samples: 68


In [14]:
# horizontally stack columns
dataset1 = hstack(s1)
# define generator
n_input = 2
n_features = dataset.shape[1]
generator1 = TimeseriesGenerator(dataset1, dataset1, length=n_input, batch_size=1)
# number of samples
print('Samples: %d' % len(generator1))

Samples: 28


In [15]:
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1000))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit_generator(generator1, steps_per_epoch=1, epochs=500, verbose=0)

Instructions for updating:
Please use Model.fit, which supports generators.


In [31]:
list = []
P = [np.array(pd.DataFrame(dataset).iloc[68]).tolist(), np.array(pd.DataFrame(dataset).iloc[69]).tolist()]
P = np.array(P)
x_input = P.reshape((1, n_input, n_features))
list.append(model.predict(x_input, verbose=0))

In [32]:
P

array([[202.84606044, 202.84590073, 207.02996792, ..., 836.35974821,
        202.8458997 , 234.58974259],
       [202.84602226, 202.8459004 , 206.54987947, ..., 827.59908027,
        202.84589967, 231.75260712]])

In [44]:
Q = [np.array(pd.DataFrame(dataset).iloc[69]).tolist(),np.array(pd.DataFrame(list[0]).iloc[0]).tolist()]
Q = np.array(Q)
x_input = Q.reshape((1, n_input, n_features))
list.append(model.predict(x_input, verbose=0))


In [46]:
for i in range(28):
  Q = [np.array(pd.DataFrame(list[i]).iloc[0]).tolist(),np.array(pd.DataFrame(list[i+1]).iloc[0]).tolist()]
  Q = np.array(Q)
  x_input = Q.reshape((1, n_input, n_features))
  list.append(model.predict(x_input, verbose=0))


In [54]:
pd.DataFrame(list[i]).iloc[0]

0      219.575394
1      216.150177
2      208.145294
3      207.208160
4      209.470612
          ...    
995    210.847687
996    206.711121
997    690.618591
998    216.392319
999    213.076965
Name: 0, Length: 1000, dtype: float32

In [74]:
df = pd.DataFrame()
for i in range(30):
  df = pd.concat([df,pd.DataFrame(list[i])],axis = 0)


In [93]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,219.575394,216.150177,208.145294,207.208160,209.470612,219.096954,209.703461,958.646545,206.195923,212.075073,220.821823,220.425568,203.014435,218.086166,209.296738,209.988632,216.990509,966.608459,220.442245,220.616531,213.163696,215.970673,211.258743,216.665298,215.309464,211.693069,223.129700,213.576080,214.902512,213.062027,212.538757,212.497360,207.071671,211.510040,208.388458,216.366745,217.167633,222.954865,222.554779,208.729187,...,211.560059,212.009079,213.460678,1016.263550,209.175949,222.847824,209.026031,213.483749,216.030777,223.572372,216.477661,213.440872,207.563980,1037.714478,224.804459,212.677917,222.837067,215.283112,215.160049,207.026520,1017.582825,209.512299,1046.453247,214.748413,212.939896,207.545288,213.003098,208.671402,496.408112,205.987213,219.361389,209.270096,214.022446,216.154587,224.065262,210.847687,206.711121,690.618591,216.392319,213.076965
0,217.409653,217.013794,215.299377,215.017960,211.628662,215.064041,212.840454,947.818970,210.883453,214.458237,217.484833,219.348572,210.381149,220.711899,214.466080,213.530273,213.343826,961.405518,213.277756,219.134689,214.477249,214.132278,213.916443,216.087479,214.539642,213.961914,216.742477,215.117035,215.406342,214.613220,214.595108,216.397720,214.225693,214.019699,217.413269,217.358292,216.139359,223.909470,219.932083,213.968430,...,212.866608,214.269043,215.379807,1014.460266,213.003052,221.783218,214.448517,215.924362,215.517868,219.848389,213.845703,217.198029,212.171097,1023.815552,220.328659,215.794815,214.507980,216.405701,216.808334,211.570374,1008.718201,212.959503,1035.398071,213.787399,214.518143,212.568893,215.071075,213.430786,496.038239,211.917465,218.925964,211.827225,213.551056,217.075394,220.251923,214.424637,209.759094,683.505798,213.097229,218.369705
0,217.514267,216.463974,216.528015,215.245346,211.659103,215.925308,213.844376,945.898254,212.337234,216.408875,214.952148,219.847046,213.105835,221.197189,214.018463,214.565414,213.245331,960.900513,212.727142,220.372375,212.700134,213.905624,213.994675,214.471573,216.236725,212.747040,213.918991,217.210297,215.815704,215.132172,215.436493,215.586395,214.948654,214.759369,216.058914,218.316910,218.010696,224.695023,220.431320,214.957428,...,212.250031,213.814240,213.406540,1013.619629,215.917206,221.403381,215.888931,217.305557,216.064529,219.383865,213.812515,217.366196,213.734726,1023.753906,219.894684,216.177399,212.833496,215.458145,215.974396,213.335388,1005.597717,215.585938,1033.764893,212.636505,213.533569,214.343918,214.870087,214.924530,494.614502,213.569946,218.177628,212.175690,213.265686,217.733109,219.456879,214.085175,210.842697,683.943054,212.222778,219.579666
0,217.456192,216.686569,216.803543,215.523895,211.666290,215.815155,214.078842,945.459351,212.448563,216.443787,214.959824,219.910965,213.179520,221.242447,214.326645,214.721237,213.136108,960.679871,212.494690,220.395676,212.909851,213.807358,214.109146,214.619919,216.110168,212.785507,213.795135,217.209579,215.928329,215.284943,215.379532,215.616776,215.215408,214.750595,216.572021,218.305038,217.855606,224.735062,220.265884,215.105118,...,212.323181,213.930832,213.671127,1013.595459,215.924301,221.294266,215.888214,217.420563,216.082138,219.271042,213.619598,217.665222,213.758698,1023.121277,219.686813,216.317245,212.611801,215.597488,216.038055,213.420242,1005.407776,215.621429,1033.433228,212.776993,213.660660,214.504013,215.131866,214.967072,494.830841,213.752518,218.110977,212.292480,213.214706,217.655243,219.301758,214.065628,210.896896,683.519958,212.213928,219.761917
0,217.449219,216.695450,216.839569,215.569839,211.688400,215.795212,214.097763,945.464233,212.464767,216.450317,214.961334,219.897476,213.218338,221.266678,214.35531

In [98]:
result = S_value.transpose() 

In [99]:
result[70:100]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
70,202.845993,202.8459,206.125009,202.8459,202.8459,202.8459,206.667216,950.347268,202.854122,202.846336,202.857517,202.8459,202.902110,220.437132,202.845901,202.8459,202.8459,957.094453,202.8459,225.665862,202.8459,202.8459,202.8459,203.260932,202.8459,202.8459,202.8459,202.845953,202.8459,202.8459,202.848943,206.458384,202.845930,202.8459,202.8459,202.8459,209.535598,232.814901,202.8459,202.8459,...,202.8459,202.8459,202.8459,980.662758,202.8459,229.619372,204.108897,202.8459,202.845904,213.239186,202.845961,202.845955,202.8459,982.582479,202.8459,202.8459,206.248635,202.8459,202.8459,202.8459,974.814913,202.8459,987.206596,202.8459,203.139853,202.8459,202.845913,202.8459,659.306396,202.845901,202.852338,202.8459,206.391019,202.845904,202.845974,206.992185,202.848414,818.532777,202.8459,229.171832
71,202.845971,202.8459,205.748978,202.8459,202.8459,202.8459,206.236255,948.089530,202.852585,202.846240,202.855391,202.8459,202.892870,218.782825,202.845900,202.8459,202.8459,955.209167,202.8459,223.591446,202.8459,202.8459,202.8459,203.202744,202.8459,202.8459,202.8459,202.845940,202.8459,202.8459,202.848339,206.048459,202.845922,202.8459,202.8459,202.8459,208.827464,230.188741,202.8459,202.8459,...,202.8459,202.8459,202.8459,979.991027,202.8459,227.236981,203.949129,202.8459,202.845903,212.195262,202.845946,202.845941,202.8459,981.999248,202.8459,202.8459,205.859988,202.8459,202.8459,202.8459,973.860838,202.8459,986.825425,202.8459,203.097402,202.8459,202.845910,202.8459,644.721926,202.845901,202.851115,202.8459,205.987911,202.845903,202.845956,206.528767,202.847909,809.166458,202.8459,226.823934
72,202.845954,202.8459,205.416151,202.8459,202.8459,202.8459,205.854005,945.740054,202.851335,202.846165,202.853655,202.8459,202.885150,217.285371,202.845900,202.8459,202.8459,953.249057,202.8459,221.707421,202.8459,202.8459,202.8459,203.152716,202.8459,202.8459,202.8459,202.845930,202.8459,202.8459,202.847855,205.685148,202.845917,202.8459,202.8459,202.8459,208.194570,227.795208,202.8459,202.8459,...,202.8459,202.8459,202.8459,979.297849,202.8459,225.068793,203.809587,202.8459,202.845902,211.256768,202.845935,202.845931,202.8459,981.398102,202.8459,202.8459,205.515819,202.8459,202.8459,202.8459,972.873516,202.8459,986.433955,202.8459,203.061082,202.8459,202.845907,202.8459,630.117016,202.845900,202.850125,202.8459,205.630735,202.845902,202.845943,206.117269,202.847506,799.507454,202.8459,224.687603
73,202.845941,202.8459,205.121546,202.8459,202.8459,202.8459,205.514937,943.295912,202.850318,202.846106,202.852236,202.8459,202.878698,215.929704,202.845900,202.8459,202.8459,951.211683,202.8459,219.996063,202.8459,202.8459,202.8459,203.109704,202.8459,202.8459,202.8459,202.845923,202.8459,202.8459,202.847467,205.363129,202.845912,202.8459,202.8459,202.8459,207.628867,225.613407,202.8459,202.8459,...,202.8459,202.8459,202.8459,978.582589,202.8459,223.095269,203.687706,202.8459,202.845901,210.412956,202.845926,202.845924,202.8459,980.778528,202.8459,202.8459,205.211019,202.8459,202.8459,202.8459,971.851898,202.8459,986.031920,202.8459,203.030008,202.8459,202.845905,202.8459,615.534692,202.845900,202.849323,202.8459,205.314235,202.845901,202.845932,205.751846,202.847184,789.564843,202.8459,222.743497
74,202.845931,202.8459,204.860759,202.8459,202.8459,202.8459,205.214156,940.754151,202.849492,202.846061,202.851076,202.8459,202.873307,214.702230,202.845900,202.8459,202.8459,949.094572,202.8459,218.441326,202.8459,202.8459,202.8459,203.072722,202.8459,202.8459,202.8459,202.845917,202.8459,202.8459,202.847156,205.077693,202.845909,202.8459,202.8459,202.8459,207.123179,223.624336,202.8459,202.8459,...,202.8459,202.8459,202.8459,977.844595,202.8459,221.298680,203.581249,202.845